# Exemplo de AES usando Python

Neste exemplo usamos criptografia, com as seguintes configurações:

* Um cipher (ou coder) é um algoritmo que executa criptografia ou descriptografia, mediante uma série de etapas bem definidas que podem ser seguidas como um procedimento.
* Utilizamos um algoritmo de criptografia denominado AES. Você ler mais na [wikipedia](https://pt.wikipedia.org/wiki/Advanced_Encryption_Standard). Logo abaixo tem uma breve descrição.
* O modo de operação para cifras de bloco é Cipher Block Chaining (CBC). Este modo de operação é criado utilizando um vetor de inicialização (tipo uma segunda chave de criptografia).

Com o AES, uma chave de um comprimento específico (por exemplo, 128, 192 e 256 bits) é utilizada para criptografar e descriptografar um bloco de mensagens. Cada cipher criptografa e descriptografa dados em blocos de "x" bits usando chaves criptográficas de tamanho pré-definido.
Neste tipo de algoritmo, as chaves de criptografia são simétricas, isto é, a mesma chave é utilizada para criptografar e descriptografar.
Assim, ambos os lados (remetente e destinario) devem conhecer e utilizar a mesma chave secreta.

Neste notebook vamos ver um exemplo de como utilizá-la.

In [ ]:
# instalar as dependências

In [1]:
!pip install cryptography

You should consider upgrading via the 'c:\users\henri\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import os

In [3]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [4]:
from cryptography.hazmat.backends import default_backend

In [5]:
backend = default_backend()

In [6]:
key = os.urandom(32)
key

b'\xb9\xed\x8d7\xb4\xbca\x8dpM=\xcf\xf2\xb1\x06m\xd0\x02\xe2\xa6>\xa12\xbb\xdfNW\xa1d\xe9v"'

In [7]:
# initialization vector 
iv = os.urandom(16)
iv

b'8.h\xd7]z\xab\\J\xa7p\x1d\xaf\x04\xf6N'

# Definimos uma mensagem simples em ASCII

In [8]:
MSG = "esta eh a mensagem que quero encriptar"

In [9]:
# criptografa

### Passo 1 - conversão

Para encriptar, não podemos passar diretamente a string. Precisamos converte-la em um array de bytes, pois o método de criptografia funciona sobre blocos de bytes.

In [10]:
b_MSG = bytearray(MSG, encoding="ascii")  # mensagem tem que ser byte array

### Passo 2 - Tentativa 1

Tentamos criptografar diretamente a mensagem

In [11]:
# 2.1 - criamos o objeto que criptografa AES com a chave gerada
aes = algorithms.AES(key)
# 2.2 - criamos o modo com o initialization vector criado na inicialização
cbc = modes.CBC(iv)
# criamos o cipher 
cipher = Cipher(aes, cbc, backend=backend)
# obtem o encriptador a partir do cipher
encryptor = cipher.encryptor()
# tentar encriptar
ct = encryptor.update(b_MSG) + encryptor.finalize()

ValueError: The length of the provided data is not a multiple of the block length.

### Passo 2 - Tentativa 2

O AES utilizando CBC exige que os dados sejam passados em blocos cujo tamanho seja múltiplo de 16 bytes.
Assim precisarmos alterar a mensagem original para que ela tenha o tamanho adequado.

In [12]:
# 2.0 adequar o tamanho da mensagem original para que seja multipla de block_size
block_size = 16
n = len(b_MSG)
spaces_add = block_size - n % block_size # calcular a qtd de espaços vamos adicionar ao final da mensagem
new_b_MSG = bytearray(MSG + ' ' * spaces_add, encoding="utf8")


# 2.1 - criamos o objeto que criptografa AES com a chave gerada
aes = algorithms.AES(key)
# 2.2 - criamos o modo com o initialization vector criado na inicialização
cbc = modes.CBC(iv)
# 2.3 - criamos o cipher 
cipher = Cipher(aes, cbc, backend=backend)
# 2.4 - obtem o encriptador a partir do cipher
encryptor = cipher.encryptor()
# tentar encriptar
ct = encryptor.update(new_b_MSG) + encryptor.finalize()

### Passo 3 - descriptografa

Na verdade este passo é feito no destinatário.
Note que no destinatário deveríamos fazer os passos 2.1 a 2.3 para criar o cipher.

In [13]:
# 3.4 - obtem o decriptador
decryptor = cipher.decryptor()

In [14]:
# 3.4 - decriptar a mensagem
decryptor.update(ct) + decryptor.finalize()

b'esta eh a mensagem que quero encriptar          '